In [ ]:
import re
import difflib
from nltk.tokenize import TweetTokenizer

In [ ]:
f = open("../data/raw/offensive/train_text.txt", 'r', encoding = "utf-8")
inputlist = [line for line in f]
f.close()

training_data, validation_data = inputlist[:len(inputlist)//2], inputlist[len(inputlist)//2:]


In [ ]:
## patterns
token_pat = re.compile(r'[\w@’]+')
skippable_pat = re.compile(r'\s+')

non_white_space = re.compile(r'[^@’\w\s]') #Finds characters that are not white_space nor word characters (nor @/’)


tokenlist = []
for i in training_data:
    tokens = []
    unmatchable = []
    line = i
    while line:
        skippable_match = re.search(skippable_pat, line)
        nws_match = re.search(non_white_space, line) #Search for non-word && non-whitespace chars (nws = non_white_space)
        if skippable_match and skippable_match.start() == 0:
            # If there is one at the beginning of the line, just skip it.
            line = line[skippable_match.end():]
        
        elif nws_match and nws_match.start() == 0: # If a character is neither non_white_space nor a word-character
            tokens.append(line[:nws_match.end()]) #Append it to tokens
            line = line[nws_match.end():] #Move further along in line
        else:
            # Else try finding a real token.
            token_match = re.search(token_pat, line)
            if token_match and token_match.start() == 0:
                # If there is one at the beginning of the line, tokenise it.
                tokens.append(line[:token_match.end()])
                line = line[token_match.end():]
            else:
                # Else there is unmatchable material here.
                # It ends where a skippable or token match starts, or at the end of the line.
                unmatchable_end = len(line)
                if skippable_match:
                    unmatchable_end = skippable_match.start()
                if token_match:
                    unmatchable_end = min(unmatchable_end, token_match.start())
                # Add it to unmatchable and discard from line.
                unmatchable.append(line[:unmatchable_end])
                line = line[unmatchable_end:]
    tokenlist.append(tokens)
    print("tokens:",tokens)
    print("unmatchable", unmatchable)


#Split on punctuation [".","!", '"' ASO], emoji's, parentheses
#multiple ........ turned into only 3[...]?


In [ ]:
#TweetTokenizer Initialisation
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
j = 0
for i in training_data: 
    temp = i
    diff = difflib.context_diff(tknzr.tokenize(i),tokenlist[j])
    #print("".join(diff), end = "")
    print(i,"tknzr:",tknzr.tokenize(i),"\ntokenlist:",tokenlist[j],"\n")
    j+=1

In [ ]:
print(training_data[-5])